# Updated Analysis

In [15]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import statsmodels.api as sm
from scipy import stats

# --- Load data ---
df = pd.read_csv("../data/tweepfake_features_cleaned.csv", sep=";")

# Keep only rows with non-missing features
feature_cols = ["V", "S", "W", "F", "C"]
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=feature_cols + ["label"])

# Binary label: 1 = bot, 0 = human
df["y"] = (df["label"].str.lower() == "bot").astype(int)

bots   = df["y"] == 1
humans = df["y"] == 0

df.head()

,text,label,V,S,W,F,C,y
0,YEA now that note GOOD,bot,1.000000,5.0,3.600000,0.400000,0.400000,1
1,Listen to This Charming Man by The Smiths,human,1.000000,8.0,4.250000,0.500000,0.000000,0
2,wish i can i would be seeing other hoes on the...,bot,0.923077,13.0,3.461538,0.538462,0.230769,1
3,The decade in the significantly easier schedul...,bot,1.000000,10.0,4.800000,0.500000,0.000000,1
4,matrix 2: pedaphile killer,human,1.000000,3.0,7.000000,0.000000,0.333333,0


---

In [7]:
# z-scores per feature
Z = (df[feature_cols] - df[feature_cols].mean()) / df[feature_cols].std(ddof=1)

# Mark a row as outlier if ANY feature has |z| > 4 (you can change threshold to 3, 4, or 5)
threshold = 4
outlier_mask = (Z.abs() > threshold).any(axis=1)

print("Number of outliers:", outlier_mask.sum())
print("Fraction of data:", outlier_mask.mean())

# Inspect a few outliers
df[outlier_mask].head()

Number of outliers: 793
Fraction of data: 0.03458351504579154


,text,label,V,S,W,F,C,y
7,nobody: NSP: penis penis penis penis penis pen...,human,0.085106,47.0,4.914894,0.0,0.042553,0
13,o7,human,0.000000,0.0,0.000000,0.0,0.000000,0
50,yes,human,1.000000,1.0,3.000000,0.0,1.000000,0
76,LMFAO,human,1.000000,1.0,5.000000,0.0,1.000000,0
106,me:,human,1.000000,1.0,2.000000,1.0,1.000000,0


In [8]:
df_no_outliers = df.loc[~outlier_mask].copy()
print("New size:", df_no_outliers.shape)

# If you want to re-use this cleaned dataset for models:
df_no_outliers.to_csv("../data/tweepfake_features_cleaned.csv", sep=";", index=False)

New size: (22137, 8)


In [9]:
def winsorize_series(s, lower=0.01, upper=0.99):
    lo, hi = s.quantile([lower, upper])
    return s.clip(lo, hi)

df_wins = df.copy()
for col in feature_cols:
    df_wins[col] = winsorize_series(df_wins[col], lower=0.01, upper=0.99)

# Check that extreme values were reduced
df[feature_cols].describe(), df_wins[feature_cols].describe()

(                  V             S             W             F             C
 count  22930.000000  22930.000000  22930.000000  22930.000000  22930.000000
 mean       0.928527     12.412828      4.272093      0.449725      0.111453
 std        0.105011      9.241268      1.041520      0.173568      0.187007
 min        0.000000      0.000000      0.000000      0.000000      0.000000
 25%        0.882353      6.000000      3.739130      0.372093      0.000000
 50%        1.000000     10.000000      4.181818      0.478261      0.050000
 75%        1.000000     17.000000      4.666667      0.555556      0.142857
 max        1.000000     57.000000     30.000000      1.000000      1.000000,
                   V             S             W             F             C
 count  22930.000000  22930.000000  22930.000000  22930.000000  22930.000000
 mean       0.930415     12.392472      4.252200      0.448418      0.111453
 std        0.094581      9.158334      0.821452      0.170028      0.18700

---

In [16]:
from statsmodels.discrete.discrete_model import Logit

uni_results = []

for feat in feature_cols:
    X = df[[feat]].values  # shape (n, 1)
    y = df["y"].values

    # Add intercept term for statsmodels
    X_sm = sm.add_constant(X)

    # Fit logistic regression
    logit_model = Logit(y, X_sm).fit(disp=False)
    
    # Predicted probabilities (in-sample, just for diagnostics)
    y_prob = logit_model.predict(X_sm)
    auc = roc_auc_score(y, y_prob)

    # McFadden pseudo-R² = 1 - (LL_model / LL_null)
    llf = logit_model.llf
    llnull = logit_model.llnull
    pseudo_r2 = 1 - llf / llnull

    uni_results.append({
        "feature": feat,
        "AUC": auc,
        "pseudo_R2": pseudo_r2,
        "coef": logit_model.params[1],         # slope for this feature
        "intercept": logit_model.params[0]
    })

uni_df = pd.DataFrame(uni_results).sort_values("AUC", ascending=False)
uni_df

,feature,AUC,pseudo_R2,coef,intercept
3,F,0.600048,1.797936e-02,2.014858,-0.907725
2,W,0.584293,1.441232e-02,-0.366163,1.567613
0,V,0.549111,1.425163e-02,-3.141517,2.945049
1,S,0.546579,1.098343e-02,0.027616,-0.330498
4,C,0.495038,3.290929e-07,-0.011405,0.018255


In [17]:
from scipy.stats import mannwhitneyu

def cohens_d(x0, x1):
    """Cohen's d where x0 = human, x1 = bot."""
    x0 = np.asarray(x0)
    x1 = np.asarray(x1)
    n0, n1 = len(x0), len(x1)
    s0, s1 = x0.std(ddof=1), x1.std(ddof=1)
    s_pooled = np.sqrt(((n0 - 1)*s0**2 + (n1 - 1)*s1**2) / (n0 + n1 - 2))
    return (x1.mean() - x0.mean()) / s_pooled

effect_rows = []

for feat in feature_cols:
    x_h = df.loc[humans, feat]
    x_b = df.loc[bots, feat]
    
    d = cohens_d(x_h, x_b)

    # Mann–Whitney U for CLES
    U, p_mw = mannwhitneyu(x_b, x_h, alternative="two-sided")
    cles = U / (len(x_b) * len(x_h))   # P( random bot value > random human value )

    effect_rows.append({
        "feature": feat,
        "mean_human": x_h.mean(),
        "mean_bot": x_b.mean(),
        "Cohen_d (bot - human)": d,
        "CLES P(bot > human)": cles,
        "MW_pvalue": p_mw
    })

effects_df = pd.DataFrame(effect_rows)
effects_df

,feature,mean_human,mean_bot,Cohen_d (bot - human),CLES P(bot > human),MW_pvalue
0,V,0.943914,0.918373,-0.281659,0.450889,3.791853e-42
1,S,11.538829,13.771874,0.247067,0.546579,3.150077e-33
2,W,4.348005,4.126324,-0.283321,0.415707,1.228280e-104
3,F,0.433097,0.483502,0.316805,0.600048,6.513939e-147
4,C,0.087598,0.087438,-0.001351,0.504962,1.846222e-01


In [18]:
X = df[feature_cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Loadings: contribution of each feature to each PC
loadings = pd.DataFrame(
    pca.components_.T,
    index=feature_cols,
    columns=[f"PC{i+1}" for i in range(len(feature_cols))]
)

explained_var = pd.Series(
    pca.explained_variance_ratio_,
    index=loadings.columns,
    name="ExplainedVarianceRatio"
)

loadings, explained_var

(        PC1       PC2       PC3       PC4       PC5
 V  0.537398  0.190912 -0.459271 -0.661062 -0.163779
 S -0.534656 -0.268341  0.328456 -0.729655 -0.043081
 W  0.241787 -0.716686  0.020922  0.133608 -0.640009
 F -0.464826  0.478755 -0.237802  0.098750 -0.698877
 C  0.388358  0.385617  0.790063 -0.054754 -0.270703,
 PC1    0.401519
 PC2    0.272579
 PC3    0.148039
 PC4    0.092914
 PC5    0.084950
 Name: ExplainedVarianceRatio, dtype: float64)

In [19]:
explained_var.cumsum()

PC1    0.401519
PC2    0.674097
PC3    0.822136
PC4    0.915050
PC5    1.000000
Name: ExplainedVarianceRatio, dtype: float64

In [20]:
def mean_diff_ci(x0, x1, alpha=0.05):
    """
    x0: humans, x1: bots
    Returns diff, (low, high), t-stat, p-val (Welch).
    """
    x0 = np.asarray(x0)
    x1 = np.asarray(x1)
    n0, n1 = len(x0), len(x1)
    
    mean0, mean1 = x0.mean(), x1.mean()
    var0, var1 = x0.var(ddof=1), x1.var(ddof=1)
    
    diff = mean1 - mean0                   # bot - human
    se = np.sqrt(var0/n0 + var1/n1)        # Welch standard error
    
    # Welch–Satterthwaite degrees of freedom
    df_num = (var0/n0 + var1/n1)**2
    df_den = (var0**2 / ((n0**2)*(n0-1))) + (var1**2 / ((n1**2)*(n1-1)))
    df_welch = df_num / df_den
    
    tcrit = stats.t.ppf(1 - alpha/2, df_welch)
    ci_low, ci_high = diff - tcrit*se, diff + tcrit*se
    
    # t-test (Welch)
    tstat, pval = stats.ttest_ind(x1, x0, equal_var=False)
    
    return diff, (ci_low, ci_high), tstat, pval, df_welch

summary_rows = []

for feat in feature_cols:
    x_h = df.loc[humans, feat]
    x_b = df.loc[bots, feat]
    
    diff, (ci_low, ci_high), tstat, pval, df_welch = mean_diff_ci(x_h, x_b)
    d = cohens_d(x_h, x_b)
    
    summary_rows.append({
        "feature": feat,
        "mean_human": x_h.mean(),
        "mean_bot": x_b.mean(),
        "mean_diff (bot - human)": diff,
        "CI_low": ci_low,
        "CI_high": ci_high,
        "Cohen_d": d,
        "t_stat": tstat,
        "df_Welch": df_welch,
        "p_value": pval
    })

summary_df = pd.DataFrame(summary_rows)
summary_df

,feature,mean_human,mean_bot,mean_diff (bot - human),CI_low,CI_high,Cohen_d,t_stat,df_Welch,p_value
0,V,0.943914,0.918373,-0.025541,-0.027924,-0.023158,-0.281659,-21.010767,20249.883391,5.662268e-97
1,S,11.538829,13.771874,2.233045,1.995447,2.470644,0.247067,18.421596,20831.097631,3.488170e-75
2,W,4.348005,4.126324,-0.221681,-0.242323,-0.201038,-0.283321,-21.049493,21554.530499,2.207173e-97
3,F,0.433097,0.483502,0.050405,0.046210,0.054600,0.316805,23.550993,21931.618805,3.906452e-121
4,C,0.087598,0.087438,-0.000160,-0.003282,0.002962,-0.001351,-0.100449,22051.359469,9.199888e-01
